In [1]:
!git clone https://github.com/ultralytics/yolov5  # clone
%cd yolov5
%pip install -qr requirements.txt comet_ml  # install

import torch
import utils
display = utils.notebook_init()  # checks

YOLOv5 🚀 v7.0-385-gb968b2d7 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)


Setup complete ✅ (2 CPUs, 12.7 GB RAM, 32.6/112.6 GB disk)


In [2]:
!unzip -q ../train_data.zip -d ../

In [3]:
# Train YOLOv5s on COCO128 for 3 epochs
!python train.py --img 640 --batch 4 --epochs 110 --data custom.yaml --weights yolov5s.pt --cache

2024-11-21 09:34:27.601118: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-21 09:34:27.636924: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-21 09:34:27.647517: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
train: weights=yolov5s.pt, cfg=, data=custom.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=110, batch_size=4, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=data/hyps, resume_evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, opti

In [41]:
!python3 detect.py --weights runs/train/exp/weights/best.pt --source /content/img1.jpg

detect: weights=['runs/train/exp/weights/best.pt'], source=/content/img1.jpg, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_format=0, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-385-gb968b2d7 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
WARNING ⚠️ NMS time limit 0.550s exceeded
image 1/1 /content/img1.jpg: 608x640 1 bus_number, 29.8ms
Speed: 0.6ms pre-process, 29.8ms inference, 635.8ms NMS per image at shape (1, 3, 640, 640)
Results saved to runs/detect/exp2


In [5]:
import torch
model = torch.hub.load('ultralytics/yolov5', 'custom', path='/content/yolov5/runs/train/exp/weights/best.pt')

Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip
YOLOv5 🚀 v7.0-385-gb968b2d7 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


In [40]:
# prompt: get me the bench marks of my above trained yolo model

!python val.py --weights runs/train/exp/weights/best.pt --data custom.yaml --img 640


val: data=/content/yolov5/data/custom.yaml, weights=['runs/train/exp/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, max_det=300, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v7.0-385-gb968b2d7 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
val: Scanning /content/train_data/labels/val.cache... 10 images, 0 backgrounds, 0 corrupt: 100% 10/10 [00:00<?, ?it/s]
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100% 1/1 [00:01<00:00,  1.71s/it]
                   all         10         10       0.99          1      0.995       0.62
Speed: 0.3ms pre-process, 13.5ms inference, 88.9ms NMS per image at shape (32, 3, 640, 640)
Results saved to runs/v

In [43]:
# Run YOLOv5 inference
results = model('/content/yolov5/runs/detect/exp2/img1.jpg')

# Access bounding box details
detections = results.xyxy[0]  # First image results (if batch inference)

for *xyxy, conf, cls in detections:  # Loop through each detection
    x1, y1, x2, y2 = map(int, xyxy)  # Bounding box coordinates (integers)
    confidence = float(conf)  # Confidence score
    class_index = int(cls)  # Class label index

    print(f"Bounding Box: ({x1}, {y1}, {x2}, {y2}), Confidence: {confidence}, Class Index: {class_index}")


Bounding Box: (274, 120, 436, 227), Confidence: 0.7547818422317505, Class Index: 0


In [45]:
# prompt: give me the image for above bounded box region and save it in files Bounding Box: (261, 112, 418, 245)

import cv2

# Assuming 'results' and 'detections' are already defined from the previous code

# Specify the desired bounding box coordinates
target_x1, target_y1, target_x2, target_y2 = 274, 120, 436, 227
# Load the image
img = cv2.imread('/content/yolov5/runs/detect/exp2/img1.jpg')

# Crop the image based on the bounding box
cropped_img = img[target_y1:target_y2, target_x1:target_x2]

# Save the cropped image
cv2.imwrite('/content/cropped_image.jpg', cropped_img)

True

In [7]:
!pip install pytesseract opencv-python-headless pyttsx3 moviepy
!apt-get install tesseract-ocr
!pip install easyocr

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 49 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 2s (2,205 kB/s)
Selecting previously unselected package tesseract-ocr-eng.
(Reading database ... 123629 files and directories currently installed.)
Preparing to unpack .../tesseract-ocr-

In [8]:
!pip install gTTS

In [13]:
!pip install paddleocr
!pip install paddlepaddle

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 5.1 MB/s eta 0:00:00
  Attempting uninstall: opt-einsum
    Found existing installation: opt_einsum 3.4.0
    Uninstalling opt_einsum-3.4.0:
      Successfully uninstalled opt_einsum-3.4.0


In [47]:
from paddleocr import PaddleOCR

# Initialize OCR model
ocr = PaddleOCR(use_angle_cls=True, lang='en')  # Set language and other options

# Perform OCR on an image
image_path = "/content/cropped_image.jpg"
result = ocr.ocr(image_path)

# Extract and print the number
for line in result[0]:
    for word_info in line:
        #word = word_info[1][0]
        if type(word_info)==tuple:  # Check if the recognized text is a number
          detected_number = word_info[0]
          print(f"Detected Number:{detected_number} Confidence: {word_info[1]}")



[2024/11/21 10:40:13] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, use_mlu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='/root/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='/root/.paddleocr/whl/rec/en/en_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=6, max_text_length=25, rec_c

In [48]:
# prompt: write a script to convert the number in variable `detected_number` into speech

from gtts import gTTS
import os

# Assuming 'detected_number' is already defined from the previous code
assistant = "The detected bus number is "+ str(detected_number)
try:
    # Create a gTTS object
    tts = gTTS(text=assistant, lang='en')

    # Save the speech to a file
    tts.save("detected_number.mp3")

    # Play the speech (optional)
    os.system("mpg321 detected_number.mp3")  # For Linux/macOS
    # os.system("start detected_number.mp3") # For Windows
    print(assistant);
    print("Speech saved as detected_number.mp3")

except NameError:
    print("Error: 'detected_number' variable is not defined.")
except Exception as e:
    print(f"An error occurred: {e}")

The detected bus number is 101
Speech saved as detected_number.mp3


In [34]:
os.system('system_info')

32512

In [49]:


from gtts import gTTS
from IPython.display import Audio, display

# Define the detected number
detected_number = detected_number  # Replace this with your detected number from OCR
assistant = f"The detected bus number is {detected_number}"

try:
    # Create a gTTS object
    tts = gTTS(text=assistant, lang='en')

    # Save the speech to a file
    tts.save("detected_number.mp3")

    # Play the speech in Colab
    print(assistant)
    print("Speech saved as detected_number.mp3")

    # Display audio playback widget
    display(Audio("detected_number.mp3", autoplay=True))

except NameError:
    print("Error: 'detected_number' variable is not defined.")
except Exception as e:
    print(f"An error occurred: {e}")


The detected bus number is 101
Speech saved as detected_number.mp3
